This notebook contains a basic workflow for generating manifest parquet files from csv files generated by our scrapers. Depending on how much RAM you have available, it may not be appropriate for really painfully huge csv files; they may require special handling to split them up.

In [1]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
import pyarrow as pa 
from pyarrow import parquet

In [2]:
usgs_buckets = ['apollo',
                'cassini',
                'chandrayaan-1',
                'clementine',
                'galileo',
                'juno',
                'lunar-orbiter',
                'lunar-reconnaissance-orbiter',
                'messenger',
                'magellan',
                'mariner-09',
                'mariner-10',
                'mariner-69',
                'mars-exploration-rover',
                'mars-global-surveyor',
                'mars-odyssey',
                'mars-pathfinder',
                'mars-reconnaissance-orbiter',
                'phoenix',
                'viking-lander',
                'viking-orbiter',
                'voyager'
                ]

In [10]:
for sub_name in usgs_buckets[1:]:
    # path you're storing csv files and manifests in
    manifest_folder = Path(os.getcwd(), "node_manifests/")
    # size-corrected scraper output csv, relative to the manifest folder
    scraper_file = f'img_manifests/img_usgs_{sub_name}_cloud_size_corrected.csv'
    # 100000 is generally a pretty good row group size for these files as 
    # a balance between loading speed and memory efficiency. If you run 
    # into a situation where you really need to optimize this, you're probably
    # better-off just splitting up the manifest files.
    row_group_size = 100000
    # name of the output manifest file you'll be creating
    manifest_filename = f'{Path(scraper_file).stem.replace("_cloud_size_corrected", "")}.parquet'

    # load scraper file into memory
    df = pd.read_csv(
        Path(manifest_folder, scraper_file), 
        header = 0, 
        dtype = {'url': str, 'size': str},
        names = ('url', 'size')
    )

    # trim missing-size entries and units column
    df['size'] = df['size'].astype(int)
    # chop off protocol string 
    df['url'] = df['url'].str.replace("http://", "")
    df['url'] = df['url'].str.replace("https://", "")
    # split url to parts by '/'; keep the first part (domain) 
    # and the last (filename) separate from the rest (url)
    fn_url = df['url'].str.rsplit('/', n=1, expand=True)
    domain_url = fn_url[0].str.split('/', n=1, expand=True)
    # reassemble all this into the search-efficient field structure 
    # used in the manifest files
    df[['domain', 'url']] = domain_url
    del domain_url
    df['filename'] = fn_url[1]
    del fn_url
    df = df.reindex(columns=['domain', 'url', 'filename', 'size'])
    
    # write that table to a parquet file
    parquet.write_table(
        pa.Table.from_pandas(df, preserve_index=False),
        Path("node_manifests", manifest_filename),
        version='2.6',
        row_group_size=row_group_size,
        use_dictionary=['domain', 'url', 'size']
    )

In [5]:
# path you're storing csv files and manifests in
manifest_folder = Path(os.getcwd(), "node_manifests/")
# size-corrected scraper output csv, relative to the manifest folder
scraper_file = f'img_manifests/img_usgs_{usgs_buckets[0]}_cloud_size_corrected.csv'
# 100000 is generally a pretty good row group size for these files as 
# a balance between loading speed and memory efficiency. If you run 
# into a situation where you really need to optimize this, you're probably
# better-off just splitting up the manifest files.
row_group_size = 100000
# name of the output manifest file you'll be creating
manifest_filename = f'{Path(scraper_file).stem.replace("_cloud_size_corrected", "")}.parquet'

In [6]:
# load scraper file into memory
df = pd.read_csv(
    Path(manifest_folder, scraper_file), 
    header = 0, 
    dtype = {'url': str, 'size': str},
    names = ('url', 'size')
)

In [7]:
# trim missing-size entries and units column
df['size'] = df['size'].astype(int)
# chop off protocol string 
df['url'] = df['url'].str.replace("http://", "")
df['url'] = df['url'].str.replace("https://", "")
# split url to parts by '/'; keep the first part (domain) 
# and the last (filename) separate from the rest (url)
fn_url = df['url'].str.rsplit('/', n=1, expand=True)
domain_url = fn_url[0].str.split('/', n=1, expand=True)
# reassemble all this into the search-efficient field structure 
# used in the manifest files
df[['domain', 'url']] = domain_url
del domain_url
df['filename'] = fn_url[1]
del fn_url
df = df.reindex(columns=['domain', 'url', 'filename', 'size'])

In [8]:
# write that table to a parquet file
parquet.write_table(
    pa.Table.from_pandas(df, preserve_index=False),
    Path("node_manifests", manifest_filename),
    version='2.6',
    row_group_size=row_group_size,
    use_dictionary=['domain', 'url', 'size']
)

### STOP HERE (unless you want to combine parquet files)

In [4]:
# should you wish to concatenate multiple parquet files
tabs = [
    parquet.read_table(f'node_manifests/img_usgs_mariner-{ds}.parquet')
    for ds in [
        '09', 
        '10', 
        '69', 
    ]
]

bigtab = pa.concat_tables(tabs)
bigname = 'img_usgs_mariner.parquet'
parquet.write_table(
    bigtab, 
    Path("node_manifests", bigname),
    row_group_size=row_group_size,
    version = "2.6",
    use_dictionary = ['domain', 'url', 'size']
)